## Load Routines for Printing Matrices

In [1]:

include("printmat.jl")

printlnPs (generic function with 1 method)

# Loading Daily S&P 500 Data

In [2]:
xx = readdlm("Data/SP500RfPs.csv",',',header=true)
x  = xx[1]
SP = convert(Array{Float64},x[:,2])      #convert to numerical array, S&P 500 level
R  = (SP[2:end]./SP[1:end-1] - 1) * 100  #returns, % 
T  = length(R)  
println("Number of days in the sample: ",T)

dN = Array{Date}(T)                      #convert to Date
for t = 1:T
    dN[t] = Date(x[t+1,1],"d/m/y")
end

Number of days in the sample: 9352


# Autocorrelations

$\rho_s = \text{Corr}(R_t,R_{t-s})$

In large samples, $\sqrt{T}\hat{\rho}_{s}\sim N(0,1)$

In [3]:
using StatsBase

plags = 1:5
ρ = autocor(R,plags)
println("\nlag, autocorrr, t-stat(autocorr) of excess returns")
printmat([plags ρ sqrt(T)*ρ])


lag, autocorrr, t-stat(autocorr) of excess returns
     1.000    -0.026    -2.480
     2.000    -0.044    -4.208
     3.000    -0.007    -0.684
     4.000    -0.022    -2.124
     5.000    -0.018    -1.744



# Autoregressions

An AR(1) is 

$R_{t}=c+a_{1}R_{t-1}+\varepsilon_{t}$

and can be estimated by OLS

An asymmteric AR(1) 

$R_{t} =\alpha+\beta Q_{t-1}R_{t-1}+\gamma(1-Q_{t-1})R_{t-1}+\varepsilon
_{t}$,

where $Q_{t-1}=1 \ \text{ if } \ R_{t-1}<0$ and zero otherwise

In [4]:
function OlsFn(y,x)                      #OLS estimation 
    T = size(y,1)                        #alternative: use GLM.jl
    b     = x\y
    yhat  = x*b
    res   = y - yhat
    Covres = cov(res)
    Covb   = kron(Covres,inv(x'x))       #works also if y is Txn
    R2a    = 1 - var(res,1)./var(y,1)
    return b,res,yhat,Covb,R2            #the outputs
end

OlsFn (generic function with 1 method)

In [5]:
y  = R[2:end]
Tb = size(y,1)
x  = [ones(Tb) R[1:end-1]]
(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1): intercept and slope with t-stats")
printmat([b tstat])

Results from AR(1): intercept and slope with t-stats
     0.039     3.405
    -0.026    -2.481



In [6]:
DummyNegative = R[1:end-1] .< 0

x = [ones(Tb) DummyNegative.*R[1:end-1] (1-DummyNegative).*R[1:end-1]]

(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt.(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats")
printmat([b tstat])

Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats
     0.001     0.058
    -0.075    -4.395
     0.027     1.518



## Recursive Estimation and Out-of-Sample R2

Do recursive estimation (longer and longer sample) an predict one period ahead (outside of the sample). Define an "out-of-sample" $R^2$ as 

$R_{OS}^2 = 1- \frac{\text{MSE(forecasting model)}}{\text{MSE(benchmark forecast)}}$

In [7]:
y = R[2:end]
Tb = size(y,1)
x = [ones(Tb,1) R[1:end-1]]

bT     = fill(NaN,size(x))
rHat   = fill(NaN,Tb)
rTilde = fill(NaN,Tb)
for t = 100:Tb
    (b,res,yhat,Covb,) = OlsFn(y[1:t-1],x[1:t-1,:]) 
    bT[t,:] = b'
    rHat[t] = (x[t:t,:]*b)[1]           #out-of-sample forecast of period t
    rTilde[t] = mean(y[1:t-1])          #benchmark forecast: historical average
end    

MSE_Model = mean((y[100:end]-rHat[100:end]).^2)    #MSE for out-of-sample period
MSE_Bench = mean((y[100:end]-rTilde[100:end]).^2)

printlnPs("MSE of AR(1) model and the benchmark: ",[MSE_Model MSE_Bench])
printlnPs("out-of-sample R2: ",1-MSE_Model/MSE_Bench)

MSE of AR(1) model and the benchmark:      1.247     1.244
out-of-sample R2:     -0.002


In [8]:
using Gadfly

YearFrac  = Dates.year(dN) + (Dates.month(dN)-1)/12 + (Dates.day(dN)-1)/(31*12)
plot1 = plot(x=YearFrac[2:end],y=bT[:,1],Geom.line,Theme(default_color=colorant"blue"),
Scale.x_continuous(minvalue=1979,maxvalue=2016),
Guide.xticks(ticks=[1980,1990,2000,2010]),
Guide.title("Estimated intercept"),
Guide.ylabel(" "))
display(plot1)

plot2 = plot(x=YearFrac[2:end],y=bT[:,2],Geom.line,Theme(default_color=colorant"red"),
Scale.x_continuous(minvalue=1979,maxvalue=2016),
Guide.xticks(ticks=[1980,1990,2000,2010]),
Guide.title("Estimated AR(1) slope"),
Guide.ylabel(" "))
display(plot2)

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 x 
 
 
 1980 
 1990 
 2000 
 2010 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path fill="none" d="M20.96,60.98 L 20.99 58.79 21 60.49 21.01 68.33 21.02 68.13 21.04 67.5 21.05 66.47 21.06 56.55 21.07 52.8 21.08 50.01 21.1 52.71 21.11 49.51 21.12 43.47 21.13 47.71 21.13 48.51 21.16 49.51 21.17 53.5 21.18 53.24 21.18 53.12 21.19 50.2 21.22 47.13 21.23 51.14 21.23 54.14 21.24 49.81 21.25 47.09 21.28 46.95 21.29 53.13 21.31 52.08 21.32 50.18 21.34 43.34 21.35 39.73 21.36 41.92 21.37 45.6 21.38 51.72 21.4 53.72 21.41 50.99 21.42 56.64 21.43 57.09 21.43 57.64 21.46 56.49 21.47 58.1 21.48 55.84 21.48 50.07 21.49 50.78 21.52 50.99 21.53 50.9 21.53 47.48 21.54 46.24 21.55 47.03 21.58 47.53 21.58 46.3 21.59 39.73 21.6 39.24 21.61 42.09 21.63 36.98 21.64 33.22 21.65 33.57 21.66 30.34 21.67 31.67 21.69 30.82 21.7 28.75 21.71 28.96 21.72 28.94 21.72 30.97 21.75 31.23 21.76 28.95 21.77 29.99 21.77 30.23 21.78 30.53 21.82 29.44 21.82 37.92 21.83 43.22 21.84 40.37 21.87 37.39 21.87 36.02 21.88 39.2 21.89 37.53 21.9 37.79 21.92 34.07 21.93 34.41 21.94 38.26 21.95 36.65 21.96 27.94 21.98 28.91 21.99 32.76 22 32.26 22.01 31.4 22.02 30.79 22.05 34.62 22.06 37.96 22.07 33.11 22.07 33.7 22.08 31.62 22.11 28.14 22.12 33.13 22.12 48.29 22.13 53.22 22.14 53.45 22.17 55.7 22.17 60.16 22.18 60.09 22.19 59.25 22.2 58.62 22.22 66.7 22.23 69.14 22.24 70.28 22.25 69.33 22.26 71.24 22.28 68.61 22.29 68.54 22.3 61.05 22.31 65.25 22.32 61.72 22.34 62.45 22.35 65.12 22.36 67.2 22.37 72.07 22.37 69.46 22.4 65.25 22.41 59.03 22.42 62.12 22.42 60.13 22.43 57.87 22.46 59.55 22.47 57.82 22.47 60.02 22.49 59.04 22.51 56.53 22.52 50.05 22.53 52.37 22.54 50.87 22.55 51.06 22.58 53.38 22.59 54.3 22.6 50.91 22.61 50.09 22.61 48.08 22.64 50.02 22.65 49.38 22.66 50.15 22.66 50.08 22.67 49.74 22.7 45.99 22.71 45.53 22.71 48.95 22.72 48.87 22.73 48.76 22.76 50.99 22.77 50.53 22.78 50.3 22.79 49.92 22.81 50.49 22.83 50.23 22.84 56.98 22.85 57.83 22.87 53.45 22.88 53.3 22.89 47.19 22.9 47.88 22.91 45.58 22.93 45.97 22.94 44.78 22.95 43.01 22.96 43.68 22.96 44.78 22.99 43.67 23 41.15 23.01 43.14 23.01 37.31 23.02 37.47 23.05 37.96 23.06 34.68 23.06 37.01 23.07 33.58 23.08 36.45 23.11 33.41 23.11 35.57 23.12 34.68 23.13 32.23 23.14 31.37 23.16 27.67 23.17 29.87 23.18 27.64 23.19 26.76 23.2 31.07 23.23 34.7 23.24 36.81 23.25 31.56 23.26 34.43 23.28 34.97 23.29 39.46 23.3 37.22 23.3 41.25 23.31 41.09 23.35 37.9 23.36 40.79 23.37 39.92 23.38 44.1 23.39 50.97 23.41 55.65 23.42 56.29 23.43 52.88 23.44 55.43 23.45 58.64 23.47 58.85 23.48 67.13 23.49 60.73 23.5 60.63 23.5 63.78 23.53 65.79 23.54 73.8 23.55 73.27 23.55 74.63 23.56 75.76 23.59 69 23.6 66.12 23.6 66.3 23.61 65.09 23.65 66.58 23.65 71.55 23.66 68.3 23.67 63.83 23.68 62.08 23.7 63.07 23.71 65.41 23.72 65.73 23.73 68.43 23.74 69.42 23.76 70.58 23.77 72.36 23.78 62.89 23.79 63.11 23.8 61.63 23.82 60.07 23.83 59.21 23.84 58.87 23.85 57.99 23.86 60.04 23.89 59.61 23.9 57.85 23.9 58.38 23.91 56.26 23.92 58.81 23.95 61.98 23.95 61.54 23.96 58.13 23.97 57.36 23.98 57.24 24 56.53 24.01 55.98 24.02 56.23 24.03 56.06 24.04 53.34 24.07 50.44 24.08 49.35 24.09 48.29 24.09 52.11 24.12 49.5 24.13 50.72 24.14 51.24 24.14 46.82 24.15 46.98 24.18 46.25 24.19 45.42 24.19 43.76 24.2 41.5 24.21 42.74 24.24 42.15 24.24 41.77 24.25 42.04 24.26 41.68 24.27 44.86 24.29 45.9 24.3 44.96 24.31 43.95 24.32 41.24 24.33 42.51 24.35 42.88 24.37 46.32 24.38 44.56 24.39 43.41 24.42 40.44 24.43 39.49 24.44 40.52 24.44 40.27 24.45 42.26 24.48 40.4 24.49 36.93 24.49 38.45 24.5 37.81 24.51 34.81 24.54 34.29 24.54 33.72 24.55 34.45 24.56 34.97 24.57 35.28 24.59 37.13 24.6 35.83 24.61 34.46 24.62 34.99 24.63 36.04 24.65 36.86 24.66 37.29 24.67 37.77 24.68 36.43 24.69 33.88 24.71 33.8 24.72 32.

<?xml version="1.0" encoding="UTF-8"?>
 
 
 
 x 
 
 
 1980 
 1990 
 2000 
 2010 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path fill="none" d="M22.06,47.98 L 22.09 47.97 22.1 48.33 22.11 47.05 22.12 47.19 22.14 47.22 22.15 47.17 22.16 46.15 22.17 41.83 22.18 40.32 22.2 41.31 22.21 42.16 22.22 40.29 22.23 43.65 22.23 43.3 22.26 43.18 22.27 42.68 22.28 42.79 22.28 42.79 22.29 42.78 22.32 41.68 22.32 43.44 22.33 42.07 22.34 43.68 22.35 42.32 22.38 42.25 22.39 42.55 22.41 43.3 22.42 43.21 22.44 41.47 22.45 37.46 22.46 39.05 22.46 38.11 22.47 35.3 22.5 33.86 22.51 34.74 22.51 36.56 22.52 36.24 22.53 36.16 22.56 36.27 22.56 36.51 22.57 37 22.58 35.28 22.59 36 22.61 35.99 22.62 36 22.63 35.99 22.64 35.3 22.65 35.51 22.67 35.46 22.68 35.58 22.69 34.39 22.7 33.73 22.7 34.38 22.73 36.53 22.74 33.49 22.74 33.81 22.75 33.74 22.76 34.54 22.79 34.7 22.79 34.47 22.8 34.55 22.81 34.55 22.82 34.55 22.84 34.46 22.85 34.67 22.86 35.1 22.87 35.07 22.88 35.05 22.91 35.12 22.92 36.77 22.93 30.41 22.93 32.61 22.96 31.6 22.97 30.65 22.98 31.88 22.98 32.77 22.99 32.82 23.02 32.94 23.02 33.21 23.03 33.23 23.04 34.26 23.05 32.69 23.07 35.37 23.08 35.02 23.09 35.33 23.1 35.33 23.11 35.24 23.14 35.76 23.15 33.61 23.16 36.37 23.16 36.89 23.17 37.01 23.2 35.52 23.21 40.27 23.21 26.83 23.22 18.77 23.23 18.58 23.26 18.12 23.26 16.34 23.27 16.39 23.28 16.51 23.29 16.43 23.31 17.5 23.32 14.44 23.33 13.75 23.34 14.04 23.35 14.23 23.37 15.08 23.38 15.05 23.39 14.41 23.4 20.83 23.4 22.83 23.43 23.2 23.44 23.04 23.44 22.07 23.45 19.99 23.46 22.26 23.49 21.05 23.49 15.69 23.5 20.26 23.51 21 23.52 20.38 23.54 21.18 23.55 21.58 23.56 22.17 23.58 22.52 23.6 22.24 23.61 18.95 23.62 22.64 23.63 23.05 23.63 23.09 23.67 23.08 23.68 22.71 23.68 23.48 23.69 22.97 23.7 22.44 23.72 23.33 23.73 23.53 23.74 23.58 23.75 23.59 23.76 23.59 23.78 23.36 23.79 22.97 23.8 23.74 23.81 23.79 23.82 23.8 23.84 23.81 23.86 24 23.86 24 23.87 23.98 23.9 24.03 23.91 24.06 23.92 24.28 23.93 23.28 23.96 24.81 23.96 24.68 23.97 23.72 23.98 24.79 23.99 24.73 24.01 24.92 24.02 24.94 24.03 24.5 24.04 24.8 24.05 24.71 24.07 24.97 24.08 24.49 24.09 25.76 24.1 27.86 24.1 28.08 24.13 28.14 24.14 28.43 24.14 30.23 24.15 31.74 24.16 33.71 24.19 35.4 24.19 36.6 24.2 36.98 24.21 36.67 24.22 36.13 24.24 35.08 24.25 37.48 24.26 38.47 24.27 38.1 24.28 39.23 24.31 36.32 24.32 34.95 24.33 37.26 24.33 40.77 24.36 40.45 24.37 39.72 24.38 41.53 24.38 42.89 24.39 43.01 24.43 43.16 24.44 45.37 24.45 45.88 24.46 46.39 24.47 40.91 24.49 35.63 24.5 35.08 24.51 35.85 24.52 37.68 24.52 36.15 24.55 36.01 24.56 35.02 24.56 44.55 24.57 44.45 24.58 44.75 24.61 43.55 24.61 40.18 24.62 40.94 24.63 40.87 24.64 40.57 24.66 42.22 24.67 38.37 24.68 38.5 24.69 38.46 24.72 38.82 24.73 37.5 24.74 40.48 24.75 38.23 24.75 36.52 24.78 36.98 24.79 36.64 24.8 36.5 24.8 36.2 24.81 35.7 24.84 35.42 24.84 34.98 24.85 38.45 24.86 38.88 24.87 38.69 24.89 38.23 24.9 37.94 24.91 37.87 24.93 37.8 24.94 38.17 24.96 38.33 24.97 38.26 24.98 38.44 24.98 38.6 24.99 39.65 25.02 38.2 25.03 38.47 25.03 38.42 25.04 37.87 25.05 37.85 25.08 37.81 25.08 37.73 25.09 37.77 25.1 37.77 25.11 37.7 25.14 35.99 25.15 35.19 25.16 34.84 25.17 35.87 25.19 37.7 25.2 38.2 25.21 38.09 25.22 38.64 25.22 38.8 25.25 38.76 25.26 38.63 25.26 38.31 25.27 37.43 25.28 38.15 25.31 38.28 25.31 38.24 25.32 38.27 25.33 38.28 25.34 38.5 25.36 37.87 25.37 38.12 25.38 37.95 25.39 37.28 25.4 38.16 25.42 38.08 25.44 37.74 25.45 38.9 25.45 38.59 25.49 37.7 25.5 36.95 25.5 37.29 25.51 37.34 25.52 37.4 25.54 38.15 25.55 36.93 25.56 38.42 25.57 38.61 25.58 38.3 25.6 37.89 25.61 37.77 25.62 37.88 25.63 37.81 25.64 37.77 25.66 37.62 25.67 38.12 25.68 37.79 25.68 37.99 25.69 37.9 25.72 37.7 25.73 37.62 25.73 37.56 25.74 37.72 25.75 36.9